In [1]:

#import libraries
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models


In [2]:

# extract frames from mp4 files and return list of frames

total_frames = 1000#control how many frames to add


def getFrames(vidfile):
    vid = cv2.VideoCapture(vidfile)
    frames = []
    endframe = True
    t = 0
    while endframe:
        if t>total_frames:
            break
        endframe,frame = vid.read()
        #print(frame)
        if endframe:
            frames.append(frame)
            #print("processing frame: ",t)
            t+=1
    return frames

#read outpub vals for txtfile and make list    
def getSpeed(txtfile):
    speeds = []
    t=0
    with open(txtfile) as f:
        for line in f :
            if t>total_frames:
                break
            speed = line.rstrip('\n')
            speed = float(speed)
            speeds.append(speed)
            #print("speed at frame: ",t,"is: ",speed)
            t+=1
    return speeds





In [3]:
#get data from files into lists

path_txt = "data/train.txt"
path_mp4 = "data/train.mp4"


images_data= getFrames(path_mp4)
speed_data = getSpeed(path_txt)

In [4]:
images_data[0].shape,speed_data[0]
#confirm image and floats in lists


((480, 640, 3), 28.105569)

[optical flow](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html)

<img src="https://opencv-python-tutroals.readthedocs.io/en/latest/_images/optical_flow_basic1.jpg">


In [5]:

#generate optical flow output with raw data

def compute_dense_optical_flow(prev_image, current_image):

  #resize images 

  scale_percent = 20 # percent of original size
  width = int(prev_image.shape[1] * scale_percent / 100)
  height = int(prev_image.shape[0] * scale_percent / 100)
  dim = (width, height)
  
  # resize image
  prev_image = cv2.resize(prev_image, dim, interpolation = cv2.INTER_AREA)
  current_image = cv2.resize(current_image, dim, interpolation = cv2.INTER_AREA) 



  old_shape = current_image.shape
  prev_image_gray = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)
  current_image_gray = cv2.cvtColor(current_image, cv2.COLOR_BGR2GRAY)

  assert current_image.shape == old_shape
  
  hsv = np.zeros_like(prev_image)
  hsv[..., 1] = 255
  flow = None
  flow = cv2.calcOpticalFlowFarneback(prev=prev_image_gray,
                                      next=current_image_gray, flow=flow,
                                      pyr_scale=0.8, levels=15, winsize=5,
                                      iterations=10, poly_n=5, poly_sigma=0,
                                      flags=10)

  mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
  hsv[..., 0] = ang * 180 / np.pi / 2
  hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

  return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR) 


In [17]:
#Create optical flow images and list of mean speed
#features and labels we will be hitting the model with

opticalFlowFrames = []
speed_Data_final=[]
for i in range(0,len(images_data)-1):#trying to get better optical flow by comparing distant frames,                                                                 #instead of consecutive 

    image = compute_dense_optical_flow(images_data[i],images_data[i+1])
    opticalFlowFrames.append(image)
    mean_speed = (speed_data[i] + speed_data[i+1])
    label = np.asarray(mean_speed,dtype= np.float32)
    speed_Data_final.append(label)


In [18]:
#see optical flow result
img = Image.fromarray(opticalFlowFrames[4], 'RGB')
img.show()


In [19]:

#train and validation split

split_ratio = 0.8
train_data = opticalFlowFrames[:int(split_ratio*len(opticalFlowFrames))]
train_labels = speed_Data_final[:int(split_ratio*len(speed_Data_final))]

val_data = opticalFlowFrames[int(split_ratio*len(opticalFlowFrames)):]
val_labels = speed_Data_final[int(split_ratio*len(speed_Data_final)):]

In [20]:
len(train_labels),len(val_labels)


(800, 200)

In [21]:
#create tensorflow datasets using our features and labels 

train_dataset = tf.data.Dataset.from_tensor_slices((train_data,train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_data,val_labels))

#batch up our datasets 
train_dataset=train_dataset.batch(10)
val_dataset=val_dataset.batch(10)


In [23]:
model = models.Sequential()
#create our model


[end to end learning for self driving cars](https://arxiv.org/pdf/1604.07316.pdf)

<img src="model.png">


In [24]:
model.add(tf.keras.layers.Conv2D(24,(5,5), strides=2, padding='same',activation='relu',input_shape=(96, 128, 3)))

#convolutions
model.add(tf.keras.layers.Conv2D(36,(5,5), strides=2, padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(48,(5,5), strides=2, padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(64,(3,3), strides=1, padding='same',activation='relu'))
model.add(tf.keras.layers.Conv2D(64,(3,3), strides=1, padding='same',activation='relu'))

model.add(tf.keras.layers.Flatten())

#dense layers
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='relu'))

In [25]:
model.summary() #observe our model

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 48, 64, 24)        1824      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 32, 36)        21636     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 16, 48)        43248     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 16, 64)        27712     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 16, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 12288)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)              

In [26]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MSE,
              metrics=['MSE'])


In [27]:
train_dataset.element_spec

(TensorSpec(shape=(None, 96, 128, 3), dtype=tf.int32, name=None),
 TensorSpec(shape=(None,), dtype=tf.float32, name=None))

In [28]:
# Define the Keras TensorBoard callback.
from datetime import datetime 
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [29]:
model.fit(train_dataset, epochs=10,validation_data=val_dataset,use_multiprocessing=True,callbacks=[tensorboard_callback])


Train for 80 steps, validate for 20 steps
Epoch 1/10
80/80 [==============================] - 27s 336ms/step - loss: 278.4848 - MSE: 278.4849 - val_loss: 490.3884 - val_MSE: 490.3884
Epoch 2/10
80/80 [==============================] - 29s 359ms/step - loss: 117.2400 - MSE: 117.2400 - val_loss: 35.4412 - val_MSE: 35.4412
Epoch 3/10
80/80 [==============================] - 32s 395ms/step - loss: 22.7735 - MSE: 22.7735 - val_loss: 13.0399 - val_MSE: 13.0399
Epoch 4/10
80/80 [==============================] - 16s 196ms/step - loss: 23.0792 - MSE: 23.0792 - val_loss: 35.5594 - val_MSE: 35.5594
Epoch 5/10
80/80 [==============================] - 11s 141ms/step - loss: 10.4027 - MSE: 10.4027 - val_loss: 36.1399 - val_MSE: 36.1399
Epoch 6/10
80/80 [==============================] - 11s 131ms/step - loss: 8.4108 - MSE: 8.4108 - val_loss: 35.4950 - val_MSE: 35.4950
Epoch 7/10
80/80 [==============================] - 11s 138ms/step - loss: 7.7273 - MSE: 7.7273 - val_loss: 33.8460 - val_MSE: 33.84

In [34]:
model.save_weights('./checkpoints/1' )